UANL  - FCFM

Segundo parcial Introducción al aprendizaje automatico

9 de Octubre de 2021

Profesor: MCE. Angel Adrián Domínguez Lozano

Alumno: Humberto Gerardo Peña Páez

# 1. **(20 puntos)** Investiga/define lo siguiente:


*   Aprendizaje supervisado
*   Conjunto de entrenamiento, validación y prueba
*   Función de pérdida
*   Parámetros e hiperparámetros
*   Regularización
*   Overfitting y underfitting
*   Variance-bias tradeoff



# 2. **(30 puntos)** Describe en tus palabras como funcionan los siguientes modelos de aprendizaje supervisado (elegir al menos 5 modelos)


*   Regresión lineal
*   Regresión logística
*   KNN
*   Decision Tree
*   Random forest
*   Gradient Boosting
*   XGBoost
*   Support Vector Machine
*   Redes neuronales

En la descripción debes mencionar/explicar parametros e hiperparámetros (maximo 4) asociados al modelo.

# 3 **(50 puntos)** Considera los datasets *titanic_train, titanic_val, titanic_test y gender_submission*





1.   Elige 3 modelos de los enlistados en el problema 2
2.   Para cada modelo elige al menos dos combinaciones de hiperparámetros distintas y entrena para predecir la supervivencia de pasajeros del titanic usando los datos *titanic_train* (es decir, debes tener al menos 6 predictores) 
3. Elige una metrica para medir el performance de tus modelos
4. Evalúa la métrica anterior sobre los datos *titanic_val*. **Define cual de todos tus predictores (son al menos 6) es el mejor**
5. Obtén las predicciones para *titanic_test* y guardalas en el mismo formato que el archivo *gender_submission*. Nombra a tu archivo **"matriculaIAAut2P.csv" y adjuntalo** como un entregable más


Tu archivo será subido a la plataforma Kaggle.(ver https://www.kaggle.com/c/titanic) el alumno con mejor puntaje en la plataforma recibirá 20 puntos extras.







##Data Reading

In [39]:
#Importar librerías necesarias.
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
label = LabelEncoder()

#Matriz de las metricas de los 6 modelos
metricMatrix =[]

#Importar Datos
data_train = pd.read_excel("/content/titanic_train.xlsx")
data_val = pd.read_excel("/content/titanic_val.xlsx")
data_test = pd.read_excel("/content/titanic_test.xlsx")

In [2]:
data_train = shuffle(data_train)
x_train = data_train
y_train = data_train['Survived']
x_train = x_train.drop(labels= ['Name', 'Ticket', 'Fare', 'Cabin', 'Survived', 'PassengerId'], axis = 1)
x_train['Family'] = x_train['SibSp'] + x_train['Parch'] + 1
x_train = x_train.drop(labels=['SibSp', 'Parch'], axis=1)
x_train['Age'] = x_train['Age'].fillna(x_train['Age'].median())
x_train['Embarked'] = x_train['Embarked'].fillna('S')
x_train['Sex'] =  label.fit_transform(x_train['Sex'])
dict_gen = {x:y for x,y in enumerate(label.classes_)}
x_train = pd.get_dummies(x_train, drop_first=True, columns=['Embarked'])

data_val = shuffle(data_val)
x_val = data_val
y_test = data_val['Survived']
x_val = x_val.drop(labels= ['Name', 'Ticket', 'Fare', 'Cabin', 'Survived', 'PassengerId'], axis = 1)
x_val['Family'] = x_val['SibSp'] + x_val['Parch'] + 1
x_val = x_val.drop(labels=['SibSp', 'Parch'], axis=1)
x_val['Age'] = x_val['Age'].fillna(x_val['Age'].median())
x_val['Embarked'] = x_val['Embarked'].fillna('S')
x_val['Sex'] =  label.fit_transform(x_val['Sex'])
dict_gen = {x:y for x,y in enumerate(label.classes_)}
x_val = pd.get_dummies(x_val, drop_first=True, columns=['Embarked'])


###Muestreo de Datos

In [3]:
x_train

,Pclass,Sex,Age,Family,Embarked_Q,Embarked_S
697,3,0,30.0,1,0,1
181,1,0,30.0,1,0,1
296,3,1,28.0,11,0,1
496,3,1,22.0,1,0,1
457,1,0,40.0,3,0,0
...,...,...,...,...,...,...
31,1,0,40.0,1,0,1
302,1,0,58.0,1,0,1
478,3,0,28.0,1,1,0
630,3,1,29.0,1,0,1


In [4]:
x_val

,Pclass,Sex,Age,Family,Embarked_Q,Embarked_S
38,3,1,30.0,1,0,0
43,3,1,29.0,1,1,0
149,2,0,4.0,3,0,1
162,3,1,18.0,1,0,1
26,1,0,19.0,3,0,1
...,...,...,...,...,...,...
21,3,1,37.0,3,0,1
119,3,0,29.0,2,1,0
49,3,0,35.0,3,0,1
24,3,1,45.0,1,0,1


In [5]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.fit_transform(x_val)

In [6]:
x_train

array([[ 0.82625875, -1.34017851,  0.07022278, -0.54793424, -0.3086067 ,
         0.62656055],
       [-1.57832283, -1.34017851,  0.07022278, -0.54793424, -0.3086067 ,
         0.62656055],
       [ 0.82625875,  0.74616926, -0.08269887,  5.33575725, -0.3086067 ,
         0.62656055],
       ...,
       [ 0.82625875, -1.34017851, -0.08269887, -0.54793424,  3.24037035,
        -1.59601494],
       [ 0.82625875,  0.74616926, -0.00623804, -0.54793424, -0.3086067 ,
         0.62656055],
       [ 0.82625875,  0.74616926, -0.61792462, -0.54793424, -0.3086067 ,
         0.62656055]])

In [7]:
x_val

array([[ 0.83212287,  0.70412948, -0.05418711, -0.66687984, -0.30335547,
        -1.74512307],
       [ 0.83212287,  0.70412948, -0.13324699, -0.66687984,  3.29646275,
        -1.74512307],
       [-0.34341579, -1.42019335, -2.10974402,  1.00501609, -0.30335547,
         0.57302549],
       ...,
       [ 0.83212287, -1.42019335,  0.3411123 ,  1.00501609, -0.30335547,
         0.57302549],
       [ 0.83212287,  0.70412948,  1.13171111, -0.66687984, -0.30335547,
         0.57302549],
       [ 0.83212287,  0.70412948, -0.5285464 , -0.66687984, -0.30335547,
         0.57302549]])

In [8]:
print(x_train.mean(axis=0).round(2),x_train.var(axis=0))
print(x_val.mean(axis=0).round(2),x_val.var(axis=0))

[-0. -0.  0. -0.  0. -0.] [1. 1. 1. 1. 1. 1.]
[ 0.  0. -0.  0.  0.  0.] [1. 1. 1. 1. 1. 1.]


##K-Nearest Neighbour 

###Modelo 1

Hiperparametros:
*   N_neighbors = 5
*   weights = uniform


In [9]:
from sklearn.neighbors import KNeighborsClassifier
knnModel_1 = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knnModel_1.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [10]:
yPred_raw = knnModel_1.predict_proba(x_val)
yPred_raw

array([[1. , 0. ],
       [0.8, 0.2],
       [0. , 1. ],
       [0.8, 0.2],
       [0. , 1. ],
       [0.8, 0.2],
       [1. , 0. ],
       [0.4, 0.6],
       [0.8, 0.2],
       [1. , 0. ],
       [0. , 1. ],
       [0.4, 0.6],
       [0.4, 0.6],
       [0.8, 0.2],
       [0. , 1. ],
       [0.6, 0.4],
       [1. , 0. ],
       [0.6, 0.4],
       [1. , 0. ],
       [0. , 1. ],
       [0.8, 0.2],
       [0.8, 0.2],
       [1. , 0. ],
       [0. , 1. ],
       [0.2, 0.8],
       [0.4, 0.6],
       [0. , 1. ],
       [0. , 1. ],
       [0.6, 0.4],
       [1. , 0. ],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.8, 0.2],
       [1. , 0. ],
       [0.8, 0.2],
       [0.6, 0.4],
       [0.8, 0.2],
       [0.6, 0.4],
       [0.8, 0.2],
       [0.2, 0.8],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.4, 0.6],
       [0. , 1. ],
       [1. , 0. ],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.2, 0.8],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.8, 0.2],
       [1. ,

In [11]:
precision_list = []
accuracy_list = []
recall_list = []
f1_list = []
cut_list = []
yPred = []
for cut in [x/1000 for x in range(1001)]:
  yPred = [1 if x[1]>= cut else 0 for x in yPred_raw]
  cut_list.append(cut)
  accuracy_list.append(accuracy_score(y_test,yPred))
  precision_list.append(precision_score(y_test,yPred))
  recall_list.append(recall_score(y_test,yPred))
  f1_list.append(f1_score(y_test,yPred))

In [12]:
df_metrics = pd.DataFrame({'Cut':cut_list,
                           'Accuracy':accuracy_list,
                           'Recall':recall_list,
                           'Precision':precision_list,
                           'F1':f1_list})
orderMetrics = df_metrics.sort_values(by=['F1'],ascending=False)
metricMatrix.append(orderMetrics['F1'].values[0])
orderMetrics.head(10)

,Cut,Accuracy,Recall,Precision,F1
500,0.500,0.814607,0.647887,0.851852,0.736
463,0.463,0.814607,0.647887,0.851852,0.736
473,0.473,0.814607,0.647887,0.851852,0.736
472,0.472,0.814607,0.647887,0.851852,0.736
471,0.471,0.814607,0.647887,0.851852,0.736
470,0.470,0.814607,0.647887,0.851852,0.736
469,0.469,0.814607,0.647887,0.851852,0.736
468,0.468,0.814607,0.647887,0.851852,0.736
467,0.467,0.814607,0.647887,0.851852,0.736
466,0.466,0.814607,0.647887,0.851852,0.736


###Modelo 2
Hiperparametros:
*   N_neighbors = 8
*   weights = distance

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knnModel_2 = KNeighborsClassifier(n_neighbors=8, weights='distance')
knnModel_2.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='distance')

In [14]:
yPred_raw = knnModel_2.predict_proba(x_val)
yPred_raw

array([[1.        , 0.        ],
       [0.875     , 0.125     ],
       [0.        , 1.        ],
       [0.75997205, 0.24002795],
       [0.07742622, 0.92257378],
       [0.60298794, 0.39701206],
       [1.        , 0.        ],
       [0.57048174, 0.42951826],
       [0.89363356, 0.10636644],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.25      , 0.75      ],
       [0.61737238, 0.38262762],
       [0.75302056, 0.24697944],
       [0.        , 1.        ],
       [0.69771627, 0.30228373],
       [0.76287113, 0.23712887],
       [0.73473206, 0.26526794],
       [0.89759298, 0.10240702],
       [0.        , 1.        ],
       [0.86270007, 0.13729993],
       [0.71415216, 0.28584784],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.09314167, 0.90685833],
       [0.4781875 , 0.5218125 ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.37312809, 0.62687191],
       [1.        , 0.        ],
       [0.

In [15]:
precision_list = []
accuracy_list = []
recall_list = []
f1_list = []
cut_list = []
yPred = []
for cut in [x/1000 for x in range(1001)]:
  yPred = [1 if x[1]>= cut else 0 for x in yPred_raw]
  cut_list.append(cut)
  accuracy_list.append(accuracy_score(y_test,yPred))
  precision_list.append(precision_score(y_test,yPred))
  recall_list.append(recall_score(y_test,yPred))
  f1_list.append(f1_score(y_test,yPred))

In [16]:
df_metrics = pd.DataFrame({'Cut':cut_list,
                           'Accuracy':accuracy_list,
                           'Recall':recall_list,
                           'Precision':precision_list,
                           'F1':f1_list})
orderMetrics = df_metrics.sort_values(by=['F1'],ascending=False)
metricMatrix.append(orderMetrics['F1'].values[0])
orderMetrics.head(10)

,Cut,Accuracy,Recall,Precision,F1
281,0.281,0.786517,0.788732,0.708861,0.746667
285,0.285,0.786517,0.788732,0.708861,0.746667
278,0.278,0.786517,0.788732,0.708861,0.746667
279,0.279,0.786517,0.788732,0.708861,0.746667
280,0.280,0.786517,0.788732,0.708861,0.746667
282,0.282,0.786517,0.788732,0.708861,0.746667
283,0.283,0.786517,0.788732,0.708861,0.746667
284,0.284,0.786517,0.788732,0.708861,0.746667
277,0.277,0.786517,0.788732,0.708861,0.746667
249,0.249,0.775281,0.816901,0.682353,0.743590


##Random Forest

###Modelo 1

Hiperparametros:

*   max_depth = 5
*   n_estimators = 100
*   min_samples_split = 2
*   min_samples_leaft = 3
*   max_features = sqrt


In [17]:
from sklearn.ensemble import RandomForestClassifier
modelRf_1 = RandomForestClassifier(max_depth=5, n_estimators=100, 
                                   min_samples_leaf= 3, min_samples_split= 2, 
                                   max_features= "sqrt")
modelRf_1.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
yPred_raw = modelRf_1.predict_proba(x_val)
yPred_raw

array([[0.83157461, 0.16842539],
       [0.84290293, 0.15709707],
       [0.1486728 , 0.8513272 ],
       [0.8363534 , 0.1636466 ],
       [0.10986857, 0.89013143],
       [0.62523373, 0.37476627],
       [0.84075506, 0.15924494],
       [0.6149123 , 0.3850877 ],
       [0.86514994, 0.13485006],
       [0.85369999, 0.14630001],
       [0.11956483, 0.88043517],
       [0.31324331, 0.68675669],
       [0.45270807, 0.54729193],
       [0.78548512, 0.21451488],
       [0.09600193, 0.90399807],
       [0.62750788, 0.37249212],
       [0.87019927, 0.12980073],
       [0.57800337, 0.42199663],
       [0.71078146, 0.28921854],
       [0.13242237, 0.86757763],
       [0.87388866, 0.12611134],
       [0.64015721, 0.35984279],
       [0.87982155, 0.12017845],
       [0.11209314, 0.88790686],
       [0.13521121, 0.86478879],
       [0.57472666, 0.42527334],
       [0.09306253, 0.90693747],
       [0.15586277, 0.84413723],
       [0.63678898, 0.36321102],
       [0.87982155, 0.12017845],
       [0.

In [19]:
precision_list = []
accuracy_list = []
recall_list = []
f1_list = []
cut_list = []
yPred = []
for cut in [x/1000 for x in range(1001)]:
  yPred = [1 if x[1]>= cut else 0 for x in yPred_raw]
  cut_list.append(cut)
  accuracy_list.append(accuracy_score(y_test,yPred))
  precision_list.append(precision_score(y_test,yPred))
  recall_list.append(recall_score(y_test,yPred))
  f1_list.append(f1_score(y_test,yPred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
df_metrics = pd.DataFrame({'Cut':cut_list,
                           'Accuracy':accuracy_list,
                           'Recall':recall_list,
                           'Precision':precision_list,
                           'F1':f1_list})
orderMetrics = df_metrics.sort_values(by=['F1'],ascending=False)
metricMatrix.append(orderMetrics['F1'].values[0])
orderMetrics.head(10)

,Cut,Accuracy,Recall,Precision,F1
380,0.380,0.848315,0.760563,0.843750,0.800000
381,0.381,0.848315,0.760563,0.843750,0.800000
382,0.382,0.848315,0.760563,0.843750,0.800000
383,0.383,0.848315,0.760563,0.843750,0.800000
384,0.384,0.848315,0.760563,0.843750,0.800000
385,0.385,0.848315,0.760563,0.843750,0.800000
399,0.399,0.848315,0.746479,0.854839,0.796992
400,0.400,0.848315,0.746479,0.854839,0.796992
401,0.401,0.848315,0.746479,0.854839,0.796992
379,0.379,0.842697,0.760563,0.830769,0.794118


###Modelo 2
Hiperparametros:

*   max_depth = 10
*   n_estimators = 1000
*   min_samples_split = 3
*   min_samples_leaft = 4
*   max_features = sqrt

In [21]:
from sklearn.ensemble import RandomForestClassifier
modelRf_2 = RandomForestClassifier(max_depth=10, n_estimators=1000, 
                                   min_samples_leaf= 4, min_samples_split= 4, 
                                   max_features= "sqrt")
modelRf_2.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
yPred_raw = modelRf_2.predict_proba(x_val)
yPred_raw

array([[0.89559587, 0.10440413],
       [0.88089514, 0.11910486],
       [0.11029844, 0.88970156],
       [0.76197352, 0.23802648],
       [0.10051264, 0.89948736],
       [0.60594888, 0.39405112],
       [0.89078762, 0.10921238],
       [0.58822356, 0.41177644],
       [0.92272862, 0.07727138],
       [0.88273572, 0.11726428],
       [0.05708123, 0.94291877],
       [0.25101259, 0.74898741],
       [0.47144781, 0.52855219],
       [0.72692114, 0.27307886],
       [0.0503784 , 0.9496216 ],
       [0.61806257, 0.38193743],
       [0.8198527 , 0.1801473 ],
       [0.52858343, 0.47141657],
       [0.71954063, 0.28045937],
       [0.07877409, 0.92122591],
       [0.83012931, 0.16987069],
       [0.68432405, 0.31567595],
       [0.96265837, 0.03734163],
       [0.12498507, 0.87501493],
       [0.07988566, 0.92011434],
       [0.47088694, 0.52911306],
       [0.06306099, 0.93693901],
       [0.0851517 , 0.9148483 ],
       [0.59603065, 0.40396935],
       [0.96630706, 0.03369294],
       [0.

In [23]:
precision_list = []
accuracy_list = []
recall_list = []
f1_list = []
cut_list = []
yPred = []
for cut in [x/1000 for x in range(1001)]:
  yPred = [1 if x[1]>= cut else 0 for x in yPred_raw]
  cut_list.append(cut)
  accuracy_list.append(accuracy_score(y_test,yPred))
  precision_list.append(precision_score(y_test,yPred))
  recall_list.append(recall_score(y_test,yPred))
  f1_list.append(f1_score(y_test,yPred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
df_metrics = pd.DataFrame({'Cut':cut_list,
                           'Accuracy':accuracy_list,
                           'Recall':recall_list,
                           'Precision':precision_list,
                           'F1':f1_list})
orderMetrics = df_metrics.sort_values(by=['F1'],ascending=False)
metricMatrix.append(orderMetrics['F1'].values[0])
orderMetrics.head(10)

,Cut,Accuracy,Recall,Precision,F1
297,0.297,0.786517,0.84507,0.689655,0.759494
288,0.288,0.786517,0.84507,0.689655,0.759494
294,0.294,0.786517,0.84507,0.689655,0.759494
293,0.293,0.786517,0.84507,0.689655,0.759494
292,0.292,0.786517,0.84507,0.689655,0.759494
291,0.291,0.786517,0.84507,0.689655,0.759494
290,0.290,0.786517,0.84507,0.689655,0.759494
289,0.289,0.786517,0.84507,0.689655,0.759494
287,0.287,0.786517,0.84507,0.689655,0.759494
298,0.298,0.786517,0.84507,0.689655,0.759494


##Support Vector Machine

###Modelo 1
Hiperparametros:


*   kernel = linear
*   probability = True
*   max_iter = 500

In [25]:
from sklearn.svm import SVC
modelSvm_1 = SVC(max_iter = 500, kernel='linear', probability=True)
modelSvm_1.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=500, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [27]:
yPred_raw = modelSvm_1.predict_proba(x_val)
yPred_raw

array([[0.78793046, 0.21206954],
       [0.77756934, 0.22243066],
       [0.19476974, 0.80523026],
       [0.73452396, 0.26547604],
       [0.20171851, 0.79828149],
       [0.74623544, 0.25376456],
       [0.7720142 , 0.2279858 ],
       [0.67232083, 0.32767917],
       [0.84304964, 0.15695036],
       [0.8341891 , 0.1658109 ],
       [0.18140555, 0.81859445],
       [0.22975792, 0.77024208],
       [0.90069959, 0.09930041],
       [0.72410424, 0.27589576],
       [0.21942605, 0.78057395],
       [0.75758294, 0.24241706],
       [0.73963857, 0.26036143],
       [0.84411204, 0.15588796],
       [0.64632472, 0.35367528],
       [0.3810304 , 0.6189696 ],
       [0.7446889 , 0.2553111 ],
       [0.70038916, 0.29961084],
       [0.8655613 , 0.1344387 ],
       [0.73414022, 0.26585978],
       [0.34813169, 0.65186831],
       [0.2122115 , 0.7877885 ],
       [0.13838476, 0.86161524],
       [0.23824658, 0.76175342],
       [0.64433964, 0.35566036],
       [0.84604499, 0.15395501],
       [0.

In [28]:
precision_list = []
accuracy_list = []
recall_list = []
f1_list = []
cut_list = []
yPred = []
for cut in [x/1000 for x in range(1001)]:
  yPred = [1 if x[1]>= cut else 0 for x in yPred_raw]
  cut_list.append(cut)
  accuracy_list.append(accuracy_score(y_test,yPred))
  precision_list.append(precision_score(y_test,yPred))
  recall_list.append(recall_score(y_test,yPred))
  f1_list.append(f1_score(y_test,yPred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
df_metrics = pd.DataFrame({'Cut':cut_list,
                           'Accuracy':accuracy_list,
                           'Recall':recall_list,
                           'Precision':precision_list,
                           'F1':f1_list})
orderMetrics = df_metrics.sort_values(by=['F1'],ascending=False)
metricMatrix.append(orderMetrics['F1'].values[0])
orderMetrics.head(10)

,Cut,Accuracy,Recall,Precision,F1
283,0.283,0.820225,0.788732,0.767123,0.777778
280,0.280,0.820225,0.788732,0.767123,0.777778
278,0.278,0.820225,0.788732,0.767123,0.777778
279,0.279,0.820225,0.788732,0.767123,0.777778
282,0.282,0.820225,0.788732,0.767123,0.777778
281,0.281,0.820225,0.788732,0.767123,0.777778
284,0.284,0.820225,0.788732,0.767123,0.777778
295,0.295,0.820225,0.774648,0.774648,0.774648
299,0.299,0.820225,0.774648,0.774648,0.774648
298,0.298,0.820225,0.774648,0.774648,0.774648


###Modelo 2
Hiperparametros:


*   kernel = rbf
*   gamma = auto
*   probability = True
*   max_iter = 500

In [30]:
from sklearn.svm import SVC
modelSvm_2 = SVC(max_iter = 500, kernel='rbf', gamma='auto', probability=True)
modelSvm_2.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=500, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [31]:
yPred_raw = modelSvm_1.predict_proba(x_val)
yPred_raw

array([[0.78793046, 0.21206954],
       [0.77756934, 0.22243066],
       [0.19476974, 0.80523026],
       [0.73452396, 0.26547604],
       [0.20171851, 0.79828149],
       [0.74623544, 0.25376456],
       [0.7720142 , 0.2279858 ],
       [0.67232083, 0.32767917],
       [0.84304964, 0.15695036],
       [0.8341891 , 0.1658109 ],
       [0.18140555, 0.81859445],
       [0.22975792, 0.77024208],
       [0.90069959, 0.09930041],
       [0.72410424, 0.27589576],
       [0.21942605, 0.78057395],
       [0.75758294, 0.24241706],
       [0.73963857, 0.26036143],
       [0.84411204, 0.15588796],
       [0.64632472, 0.35367528],
       [0.3810304 , 0.6189696 ],
       [0.7446889 , 0.2553111 ],
       [0.70038916, 0.29961084],
       [0.8655613 , 0.1344387 ],
       [0.73414022, 0.26585978],
       [0.34813169, 0.65186831],
       [0.2122115 , 0.7877885 ],
       [0.13838476, 0.86161524],
       [0.23824658, 0.76175342],
       [0.64433964, 0.35566036],
       [0.84604499, 0.15395501],
       [0.

In [32]:
precision_list = []
accuracy_list = []
recall_list = []
f1_list = []
cut_list = []
yPred = []
for cut in [x/1000 for x in range(1001)]:
  yPred = [1 if x[1]>= cut else 0 for x in yPred_raw]
  cut_list.append(cut)
  accuracy_list.append(accuracy_score(y_test,yPred))
  precision_list.append(precision_score(y_test,yPred))
  recall_list.append(recall_score(y_test,yPred))
  f1_list.append(f1_score(y_test,yPred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
df_metrics = pd.DataFrame({'Cut':cut_list,
                           'Accuracy':accuracy_list,
                           'Recall':recall_list,
                           'Precision':precision_list,
                           'F1':f1_list})
orderMetrics = df_metrics.sort_values(by=['F1'],ascending=False)
metricMatrix.append(orderMetrics['F1'].values[0])
orderMetrics.head(10)

,Cut,Accuracy,Recall,Precision,F1
283,0.283,0.820225,0.788732,0.767123,0.777778
280,0.280,0.820225,0.788732,0.767123,0.777778
278,0.278,0.820225,0.788732,0.767123,0.777778
279,0.279,0.820225,0.788732,0.767123,0.777778
282,0.282,0.820225,0.788732,0.767123,0.777778
281,0.281,0.820225,0.788732,0.767123,0.777778
284,0.284,0.820225,0.788732,0.767123,0.777778
295,0.295,0.820225,0.774648,0.774648,0.774648
299,0.299,0.820225,0.774648,0.774648,0.774648
298,0.298,0.820225,0.774648,0.774648,0.774648


##Manipulación de datos

Métrica a evaluar: F1.

In [34]:
metricMatrix

[0.7360000000000001,
 0.7466666666666666,
 0.8,
 0.759493670886076,
 0.7777777777777778,
 0.7777777777777778]

El modelo con mejor métrica es el 0.80, que es el tercer modelo, es decir, Random Forest 1.

In [42]:
final_pred = modelSvm_1.predict(x_val)
final_pred

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0])

In [47]:
finalPred_df = pd.DataFrame({'Passenger Id': data_val['PassengerId'], 'Survived':final_pred})
finalPred_df

,Passenger Id,Survived
0,2,0
1,4,0
2,13,1
3,28,0
4,30,1
...,...,...
173,875,0
174,878,1
175,888,1
176,889,0


In [50]:
finalPred_df.to_csv('1862464IAAut2P.csv',index=False)